In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [52]:
boat_review_df = pd.read_csv("/home/raghu/Downloads/Datasets/boat_product/BoatProduct.csv")

In [53]:
boat_review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ProductName      1449 non-null   object
 1   ProductPrice     1448 non-null   object
 2   Discount         1448 non-null   object
 3   NumberofReviews  1448 non-null   object
 4   Rate             1448 non-null   object
 5   Review           874 non-null    object
 6   Summary          1448 non-null   object
dtypes: object(7)
memory usage: 79.4+ KB


In [54]:
boat_review_df.head()

,ProductName,ProductPrice,Discount,NumberofReviews,Rate,Review,Summary
0,Stone 1000v2,"\nSale price₹ 3,999",43% off,7 reviews,★\n 5.0\n,Fantastic product,Very good product in this price range..
1,Stone 1000v2,"Sale price₹ 3,999",43% off,7 reviews,★\n 5.0\n,Outstanding,Outstanding Product
2,Stone 1000v2,"\nSale price₹ 3,999",43% off,7 reviews,★\n 5.0\n,Awesome,Works like a charm. Good sound quality. Visual...
3,Stone 1000v2,"\nSale price₹ 3,999",43% off,7 reviews,★\n 5.0\n,Boat stone 1000 v2,Great
4,Stone 1000v2,"\nSale price₹ 3,999",43% off,7 reviews,★\n 5.0\n,Quality sound,Amazing sound quality.totally premium.&.afford...


In [55]:
boat_review_df.isna().sum()

ProductName          0
ProductPrice         1
Discount             1
NumberofReviews      1
Rate                 1
Review             575
Summary              1
dtype: int64

In [56]:
boat_review_df = boat_review_df.dropna()

In [57]:
boat_review_df.head()
boat_review_df.drop(['ProductName'],axis=1,inplace=True)

In [58]:
new = boat_review_df['ProductPrice'].str.split(" ",n = 1, expand = True)
boat_review_df['ProductPrice'] = new[1]
boat_review_df['ProductPrice'] = boat_review_df["ProductPrice"].str.replace("price₹",'')
boat_review_df['ProductPrice'] = boat_review_df['ProductPrice'].str.replace(',','').astype("float64")

In [59]:
boat_review_df['ProductPrice']
boat_review_df['NumberofReviews'] = boat_review_df['NumberofReviews'].str.split(" ", n=1, expand = True)[0]
boat_review_df['Discount'] = boat_review_df['Discount'].str.split("%",n = 1, expand = True)[0]
boat_review_df['Rate'] = boat_review_df['Rate'].str.split(" ",n=1,expand = True)[1]

In [60]:
my_dict = {'name':['Ravi@King','Raju@Queen','alex@jack']}
df = pd.DataFrame(data = my_dict)
print(df.name.str.split('',n=1,expand=True)[1])

0     Ravi@King
1    Raju@Queen
2     alex@jack
Name: 1, dtype: object


In [61]:
boat_review_df.head()

,ProductPrice,Discount,NumberofReviews,Rate,Review,Summary
0,3999.0,43,7,5.0\n,Fantastic product,Very good product in this price range..
1,3999.0,43,7,5.0\n,Outstanding,Outstanding Product
2,3999.0,43,7,5.0\n,Awesome,Works like a charm. Good sound quality. Visual...
3,3999.0,43,7,5.0\n,Boat stone 1000 v2,Great
4,3999.0,43,7,5.0\n,Quality sound,Amazing sound quality.totally premium.&.afford...


In [62]:
import string
import spacy
import re
nlp = spacy.load("en_core_web_sm")
from spacy.lang.en.stop_words import STOP_WORDS

In [63]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS
import spacy
from collections import Counter 

nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package wordnet to /home/raghu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/raghu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/raghu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [65]:
def preprocess(text,remove_digits = True):
    #command to replace all the non-word characters
    #all the character other then english alphabet and numeric[0-9] are termed as non_word characters
    text = re.sub("\W+",' ',text)

    #for replacing extra spaces
    text = re.sub('\s+',' ',text) 

    #for replacing all numbers ecept those attached to a word
    text = re.sub("(?<!\w)\d+", "",text)
    text = re.sub("-(?!\w)|(?<!\w)-", "",text)

    #removing all the hyphen expect the words
    text = text.lower()
    #it convert text into lower case

    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)

    return [word for word in nopunc.split() if word.lower() not in STOP_WORDS]

In [66]:
#lemmatize a word means reducing the different form of words into one form
#like 'builds','building','built' = build
def lemmatize_verbs(words):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas
def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

In [67]:
def listtostring(s):
    str1=''
    return (str1.join(s))

In [68]:
def clean_text(input):
    word = preprocess(input)
    lemmas = lemmatize(word)
    return listtostring(lemmas)

In [76]:
boat_review_df['Summary'] = boat_review_df['Summary'].astype('category')   
boat_review_df['clean'] = boat_review_df['Summary'].apply(clean_text)  
boat_review_df['Discount'] = boat_review_df['Discount'].astype('category')  

In [78]:
boat_review_df.head() 

,ProductPrice,Discount,NumberofReviews,Rate,Review,Summary,clean,Polarity,Analysis
0,3999.0,43,7,5.0\n,Fantastic product,Very good product in this price range..,goodproductpricerange,0.0,Negative
1,3999.0,43,7,5.0\n,Outstanding,Outstanding Product,outstandingproduct,0.0,Negative
2,3999.0,43,7,5.0\n,Awesome,Works like a charm. Good sound quality. Visual...,worklikecharmgoodsoundqualityvisuallyattractivev1,0.0,Negative
3,3999.0,43,7,5.0\n,Boat stone 1000 v2,Great,great,0.8,Positive
4,3999.0,43,7,5.0\n,Quality sound,Amazing sound quality.totally premium.&.afford...,amazesoundqualitytotallypremiumaffordable,0.0,Negative


In [71]:
boat_review_df['clean'] 

0                                   goodproductpricerange
1                                      outstandingproduct
2       worklikecharmgoodsoundqualityvisuallyattractivev1
3                                                   great
4               amazesoundqualitytotallypremiumaffordable
                              ...                        
1442    definitelyworthmoneysimilarpriceswaybetterskul...
1444     monthsfullysatisfyrockerzproawesomebatterybackup
1445                         soundqualitygoodbestmaterial
1446                      bestservicebestproductstaisfied
1447    haventhearfriendssaytuearphoneshatakebaatkarbc...
Name: clean, Length: 874, dtype: object

In [72]:
from textblob import TextBlob
def getPolarity(review):
    return TextBlob(review).sentiment.polarity

def analysis(score):
    if score <= 0:
        return 'Negative'   
    elif score >= 0.6:
        return 'Positive'   
    else:
        return 'Neutral'   

In [73]:
boat_review_df['Polarity'] = boat_review_df['clean'].apply(getPolarity)
boat_review_df['Analysis'] = boat_review_df['Polarity'].apply(analysis)
boat_review_df.head()    

,ProductPrice,Discount,NumberofReviews,Rate,Review,Summary,clean,Polarity,Analysis
0,3999.0,43,7,5.0\n,Fantastic product,Very good product in this price range..,goodproductpricerange,0.0,Negative
1,3999.0,43,7,5.0\n,Outstanding,Outstanding Product,outstandingproduct,0.0,Negative
2,3999.0,43,7,5.0\n,Awesome,Works like a charm. Good sound quality. Visual...,worklikecharmgoodsoundqualityvisuallyattractivev1,0.0,Negative
3,3999.0,43,7,5.0\n,Boat stone 1000 v2,Great,great,0.8,Positive
4,3999.0,43,7,5.0\n,Quality sound,Amazing sound quality.totally premium.&.afford...,amazesoundqualitytotallypremiumaffordable,0.0,Negative


In [74]:
from sklearn.model_selection import train_test_split
x = boat_review_df['clean']
y = boat_review_df['Analysis']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3, random_state = 1)

In [75]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)

ValueError: could not convert string to float: 'workgoodnice'

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
text = ['john is a good boy. John watches basketball']
vectorizer = CountVectorizer()
vectorizer.fit(text)
print(vectorizer.vocabulary_)
vector = vectorizer.transform(text)
print(vector.shape)
print(vector.toarray()) 

{'john': 4, 'is': 3, 'good': 2, 'boy': 1, 'watches': 5, 'basketball': 0}
(1, 6)
[[1 1 1 1 2 1]]
